In [1]:
import numpy as np
import pandas as pd
import scipy.io
from scipy import linalg as LA
from sklearn.ensemble import RandomForestClassifier

In [2]:
def covariance(X):
    return np.dot(X,X.T)/np.trace(np.dot(X,X.T))

def get_feat(data,sf):
    return np.log(np.var(np.dot(sf.T,data.T),axis=1)) #check axis

In [3]:
def get_spatial(left, right, J):
    c = left + right
    eigvals,eigvecs = LA.eig(c)
    diag_inv = np.zeros((c.shape[1],c.shape[1]))
    for i in range(eigvals.shape[0]):
        diag_inv[i,i]=(1/np.abs(eigvals[i].real))
        
    P = np.sqrt(diag_inv)*eigvecs.T
    s_l = P* left*P.T
    s_r = P* right*P.T
    E1, U1 = LA.eig(s_l,s_r)
    ord1 = np.argsort(E1)
    ord1 = ord1[::-1]
    E1 = E1[ord1]
    U1 = U1[:,ord1]
    W = np.dot(U1,P)
    W_select = np.zeros([np.shape(W)[0],2*J])
    W_select[:,0:J]=W[:,0:J]
    W_select[:,J::] = W[:,np.shape(W)[1]-J:np.shape(W)[1]]
    return W_select

In [5]:
num_sub=14
features_test=dict()
features_train=dict()
labels_train=dict()
labels_test=dict()
num_channels=15
hand=np.zeros((num_channels,num_channels))
leg=np.zeros((num_channels,num_channels))
labels=[]
spatial_filter=dict()
for sub in range(num_sub):
    features_train[sub]=[]
    labels_train[sub]=[]
    file_train = 'S0%dT.mat'% (sub+1)
#     mat_train = scipy.io.loadmat(file_train)
    if (sub+1)<10:    
        file_train='S0%dT.mat'% (sub+1)
        mat_train=scipy.io.loadmat(file_train)
    else:
        file_train='S%dT.mat'% (sub+1)
        mat_train=scipy.io.loadmat(file_train)
    data_train=mat_train['data']
    for k in range(5):
        cell_train=data_train[0][k]
        X_train=cell_train[0][0][0]
        time_train=cell_train[0][0][1]
        labels_x_train=cell_train[0][0][2]
        labels_train[sub].extend(labels_x_train[0])
        var=0
        for l_x in range(len(labels_x_train[0])):
            if labels_x_train[0][l_x]==1:
                hand+=covariance(X_train[var:time_train[0][l_x],:].T) #transpose because we need num_channel vs num_channel
            else:
                leg+=covariance(X_train[var:time_train[0][l_x],:].T)
            var=time_train[0][l_x]
        mean_hand=hand/num_sub
        mean_leg=leg/num_sub
        num_features=3 
        spatial_filter[sub]=get_spatial(mean_hand,mean_leg,num_features)
    
        var=0
        for count in range(len(time_train[0])):
            features_train[sub].append(get_feat(X_train[var:time_train[0][count],:],spatial_filter[sub]))
            var=time_train[0][count]    
    features_train[sub]=np.array(features_train[sub]) 
    np.random.shuffle(features_train[sub])
print("Training Features Computed\n Computing Testing features...")
for sub in range(num_sub): 
    features_test[sub]=[]
    labels_test[sub]=[]
    file_test = 'S0%dE.mat'% (sub+1)
#     mat_test = scipy.io.loadmat(file_train)
    if (sub+1)<10:
        file_test='S0%dE.mat'% (sub+1)
        mat_test=scipy.io.loadmat(file_test)
    else:
        file_test='S%dE.mat'% (sub+1)
        mat_test=scipy.io.loadmat(file_test)
    data_test=mat_test['data']
    for k in range(3):    
        cell_test=data_test[0][k]
        X_test=cell_test[0][0][0]
        time_test=cell_test[0][0][1]
        labels_x_test=cell_test[0][0][2]
        labels_test[sub].extend(labels_x_test[0])    
        var=0
        for count in range(len(time_test[0])):
            features_test[sub].append(get_feat(X_test[var:time_test[0][count],:],spatial_filter[sub]))
            var=time_test[0][count]   
    features_test[sub]=np.array(features_test[sub])
    np.random.shuffle(features_test[sub])
print("Features for training and Testing computed")

Training Features Computed
 Computing Testing features...
Features for training and Testing computed


In [6]:
labels_tr = np.array(labels_train[sub])
labels_ts = np.array(labels_test[sub])
features_tr = features_train[sub]
features_ts = features_test[sub]


In [6]:
model=RandomForestClassifier(max_depth=None,min_samples_leaf=1,random_state=0,n_estimators=1500)
model.fit(features_tr,labels_tr)
pred_train=model.predict(features_tr)
pred_test=model.predict(features_ts)

print('Training Features Shape:', features_tr.shape)
print('Training Labels Shape:', labels_tr.shape)
print('Testing Features Shape:', features_ts.shape)
print('Testing Labels Shape:', labels_ts.shape)


Training Features Shape: (100, 6)
Training Labels Shape: (100,)
Testing Features Shape: (60, 6)
Testing Labels Shape: (60,)


In [7]:
#calculate accuracy
score_tr = 0
for i in range(len(pred_train)):
    if(pred_train[i] == labels_tr[i]):
        score_tr+=1
accuracy_tr = (score_tr/len(pred_train)*100)
print('Training Accuracy: ', round(accuracy_tr,2))
score_ts = 0
for i in range(len(pred_test)):
    if (pred_test[i]==labels_ts[i]):
        score_ts+=1
accuracy = (score_ts/len(pred_test) *100)
print('Testing Accuracy: ',round(accuracy,2))

Training Accuracy:  100.0
Testing Accuracy:  70.0
